In [1]:
%%capture
## Loading Enviromet

## packages installation for running notebook
! pip install nglview
! pip install ipywidgets==7.0.0
! pip install biopython


! pip install "biobb_io==3.8.0"
! pip install "biobb_structure_utils>=3.8.0"
! pip install "biobb_vs>=3.8.1"


In [9]:
# import drive (it may ask for permission)
from google.colab import drive
drive.mount('/content/drive')

PATHWDIR = '/content/drive/MyDrive/MolDock4Colab'
print(PATHWDIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/MolDock4Colab


# 1. Molecular Docking: Preparing Protein Structure and Box

We have learned the fundamentals of Molecular Docking and will conduct a Virtual Screening for a specific protein. However, as this is for educational purposes, certain aspects should be approached with more caution.

We will be using BIOBB and SMINA, and for generating the chemical library, we will use Data Warrior, an open-source software. 

Notebook outline:

    1.1. Download and Visualize
    1.2. Preparation of receptor
    1.3. Find Binding Cavity

The term "receptor" is used to refer to the molecular system, typically a protein, that serves as the target in docking experiments. 
Prior to conducting molecular docking experiments, we need to know more about our protein. Go ahead and include the following data in your presentation, at least the questions in bold.

- **Name, including abbreviations.**
- **Functions that it performs.**
- **Primary structure and number of residues.**
- **Secondary structure.**
- **Tertiary structure.**
- **Quaternary structure (monomer, dimer...).**
- **Known active site, or other structural features important for the function.**
- Important reference articles.
- In which organisms it is found and in which tissues or cellular compartments.


For this tutorial, I am using the protein 2BRC.

Hsp90 (Heat-shock protein 90) is a type of molecular chaperone that plays a crucial role in the ATP-dependent folding, stability, and activity of numerous "client" proteins that participate in the advancement and advancement of cancer.
 [Go to article](https://www.sciencedirect.com/science/article/pii/S0960894X05006232?via%3Dihub).

## 1.1. Download and Visualization

It is useful to have a representative structure of the receptor, for example, a protein in cartoon/ribbon representation with an indication of the active site. We could do this manually, but it has already been done in previous lessons (with Pymol). 

Now we are going to use Python! 

In [18]:
# you need: ! pip install jupyterlab
import nglview as nv
print(f'nglview version: {nv.__version__}')
import ipywidgets
print(f'ipywidwgets version: {ipywidgets.__version__}')
assert ipywidgets.__version__ == '7.0.0', "You'll have problems here"

from google.colab import output
output.enable_custom_widget_manager()

nglview version: 3.0.3
ipywidwgets version: 7.0.0


#### 1.1.1 Download Protein using biobb_io

In [10]:
import os

PATH = os.path.join(PATHWDIR, 'Output') # your output path

if not os.path.exists(PATH):
   os.makedirs(PATH)
   print('Created output folder')

# This are example protein, you can change here
pdb_code = "2BRC"         # 2BRC + Hsp90 Inhibitor
ligand_code = "CT5"       # Hsp90 Inhibitor

Created output folder


In [11]:
from biobb_io.api.pdb import pdb

download_pdb = os.path.join(PATH, "download.pdb") # Name of the output file

prop = {
  "pdb_code": pdb_code, # our pdb code
  "filter": ["ATOM", "HETATM"] # what we want to retrieve 
}

pdb(output_pdb_path=download_pdb,
    properties=prop)

2023-04-19 11:23:29,125 [MainThread  ] [INFO ]  Executing biobb_io.api.pdb Version: 3.8.0


INFO:/content/log3.out:Executing biobb_io.api.pdb Version: 3.8.0


2023-04-19 11:23:29,129 [MainThread  ] [INFO ]  Downloading 2brc from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb2brc.ent


INFO:/content/log3.out:Downloading 2brc from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb2brc.ent


2023-04-19 11:23:30,147 [MainThread  ] [INFO ]  Writting pdb to: /content/drive/MyDrive/MolDock4Colab/Output/download.pdb


INFO:/content/log3.out:Writting pdb to: /content/drive/MyDrive/MolDock4Colab/Output/download.pdb


2023-04-19 11:23:30,154 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'HETATM']


INFO:/content/log3.out:Filtering lines NOT starting with one of these words: ['ATOM', 'HETATM']


0

Is interesiting to always visualize our protein before we start our experiments. 

Note: the following code dont show crystalographic waters, but they are there!!! (you can check this with Pymol if you don't trust me).

In [12]:
view = nv.show_structure_file(download_pdb, default=True)
view.center()
view._remote_call('setSize', target='Widget', args=['600px','400px'])
view

A Jupyter Widget

#### 1.1.2. Extract Protein Structure

For performing our experiment, we need to have our receptor alone (only the protein, no waters, ligands or molecules residual from crystalography). 
We remove these this in the next chunk of code, it will also print some information about the deleted reidues. The new file is written in pdb_protein.pdb.

In [13]:
from biobb_structure_utils.utils.extract_molecule import extract_molecule

pdb_protein = os.path.join(PATH, "pdb_protein.pdb")

extract_molecule(input_structure_path=download_pdb,
                 output_molecule_path = pdb_protein)

2023-04-19 11:26:07,929 [MainThread  ] [INFO ]  Executing biobb_structure_utils.utils.extract_molecule Version: 4.0.0


INFO:/content/log4.out:Executing biobb_structure_utils.utils.extract_molecule Version: 4.0.0


2023-04-19 11:26:07,944 [MainThread  ] [INFO ]  Copy: /content/drive/MyDrive/MolDock4Colab/Output/download.pdb to /content/fdb48eb3-08f5-4688-92db-da3d97d561bd


INFO:/content/log4.out:Copy: /content/drive/MyDrive/MolDock4Colab/Output/download.pdb to /content/fdb48eb3-08f5-4688-92db-da3d97d561bd


2023-04-19 11:26:07,948 [MainThread  ] [INFO ]  Creating 14249fb4-2795-4b1f-a0a1-37781ce1df76 temporary folder


...

2023-04-19 11:26:08,786 [MainThread  ] [INFO ]  ================================================================================
=                   BioBB structure checking utility v3.12.1                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-23            =

Structure /content/drive/MyDrive/MolDock4Colab/Output/download.pdb loaded
 PDB id: 
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  473
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  259
 Num. ligands or modified residues:  1
 Num. water mol.:  258
 Num. atoms:  1948
Small mol ligands found
CT5 A1215

Step 1: ligands --remove All

Running ligands. Options: --remove All
1 Ligands detected
 CT5 A1215
Ligands removed All (1)

Step 2: water --remove Yes

Running water. Options: --remove Yes
258 Water molecules detected
258 Water molecules removed
Command list completed
Final Num. models

INFO:/content/log4.out:================================================================================
=                   BioBB structure checking utility v3.12.1                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-23            =

Structure /content/drive/MyDrive/MolDock4Colab/Output/download.pdb loaded
 PDB id: 
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  473
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  259
 Num. ligands or modified residues:  1
 Num. water mol.:  258
 Num. atoms:  1948
Small mol ligands found
CT5 A1215

Step 1: ligands --remove All

Running ligands. Options: --remove All
1 Ligands detected
 CT5 A1215
Ligands removed All (1)

Step 2: water --remove Yes

Running water. Options: --remove Yes
258 Water molecules detected
258 Water molecules removed
Command list completed
Final Num. models: 1
Final Num. chains: 1 

2023-04-19 11:26:08,789 [MainThread  ] [INFO ]  Removed: ['/content/fdb48eb3-08f5-4688-92db-da3d97d561bd', '14249fb4-2795-4b1f-a0a1-37781ce1df76']


INFO:/content/log4.out:Removed: ['/content/fdb48eb3-08f5-4688-92db-da3d97d561bd', '14249fb4-2795-4b1f-a0a1-37781ce1df76']


0

We can also extract ligand in a new file:


In [19]:
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms

pdb_ligand = os.path.join(PATH, 'pdb_ligand.pdb')

prop = {
    'heteroatoms': [
        {
            'name': ligand_code
        }
    ]
}
extract_heteroatoms(input_structure_path=download_pdb,
                    output_heteroatom_path=pdb_ligand,
                    properties=prop)

2023-04-19 11:38:26,206 [MainThread  ] [INFO ]  Executing biobb_structure_utils.utils.extract_heteroatoms Version: 4.0.0


INFO:/content/log7.out:Executing biobb_structure_utils.utils.extract_heteroatoms Version: 4.0.0


2023-04-19 11:38:26,217 [MainThread  ] [INFO ]  Copy: /content/drive/MyDrive/MolDock4Colab/Output/download.pdb to /content/529b029d-5d59-4040-a760-43d326ede24c


INFO:/content/log7.out:Copy: /content/drive/MyDrive/MolDock4Colab/Output/download.pdb to /content/529b029d-5d59-4040-a760-43d326ede24c


2023-04-19 11:38:26,459 [MainThread  ] [INFO ]  Writting pdb to: /content/529b029d-5d59-4040-a760-43d326ede24c/pdb_ligand.pdb


INFO:/content/log7.out:Writting pdb to: /content/529b029d-5d59-4040-a760-43d326ede24c/pdb_ligand.pdb


2023-04-19 11:38:26,482 [MainThread  ] [INFO ]  Removed: ['/content/529b029d-5d59-4040-a760-43d326ede24c']


INFO:/content/log7.out:Removed: ['/content/529b029d-5d59-4040-a760-43d326ede24c']


0

Now we can visualize both again, and check that we have removed the ligand.

In [15]:
# change pdb_protein to pdb_ligand to visualize the ligand
view = nv.show_structure_file(pdb_protein, default=True)
view.center()
view._remote_call('setSize', target='Widget', args=['400px','400px'])
view

A Jupyter Widget

## 1.2. Preparation of receptor

Once we have the receptor alone in a new pdb file, we need to further prepare it for the molecular docking experiment. 

This includes add polar waters and charges (for a more realistic simulation), and we will save this in a .pdbqt format.

We are using the package biobb_model. However, there are different available software to do this, we could have used for example AutoDockTools (only for Windows and Linux).



We can check the structure before running the check_add_hydrogens, if you want to do this, uncoment the following chunk of code.

In [16]:
#! pip install "biobb_model>=3.8.0"

#from biobb_structure_utils.utils.structure_check import structure_check

#summary_path = os.path.join(PATH, 'check_structure.json' )

#prop = {
#        'features': ['models', 'chains', 'ligands', 'backbone']
#}

#structure_check(input_structure_path=pdb_protein,
#                output_summary_path= summary_path,
#                properties=prop)

However, the function *str_check_add_hydrogens* performs a structure check at the same time it adds hydrogens.

If we selected the pH mode, it will add hydrogens appropriate for pH ([Documentation](https://biobb-structure-utils.readthedocs.io/en/latest/command_line.html#str-check-add-hydrogens) ).

In [20]:
from biobb_structure_utils.utils.str_check_add_hydrogens import str_check_add_hydrogens

prep_receptor = os.path.join(PATH, "prep_receptor.pdbqt")
prop = {
    "charges": True, # if true, output is in pdbqt format
    "mode": "auto" # options: auto, list, pH
}



str_check_add_hydrogens(
            input_structure_path = pdb_protein,
            output_structure_path = prep_receptor,
            properties=prop)


2023-04-19 11:38:42,284 [MainThread  ] [INFO ]  Executing biobb_structure_utils.utils.str_check_add_hydrogens Version: 4.0.0


INFO:/content/log8.out:Executing biobb_structure_utils.utils.str_check_add_hydrogens Version: 4.0.0


2023-04-19 11:38:42,292 [MainThread  ] [INFO ]  Copy: /content/drive/MyDrive/MolDock4Colab/Output/pdb_protein.pdb to /content/94a5f142-c6d7-4b60-a3f8-5cf5a45d6ca7


INFO:/content/log8.out:Copy: /content/drive/MyDrive/MolDock4Colab/Output/pdb_protein.pdb to /content/94a5f142-c6d7-4b60-a3f8-5cf5a45d6ca7


2023-04-19 11:38:42,296 [MainThread  ] [INFO ]  check_structure -i /content/94a5f142-c6d7-4b60-a3f8-5cf5a45d6ca7/pdb_protein.pdb -o /content/94a5f142-c6d7-4b60-a3f8-5cf5a45d6ca7/prep_receptor.pdbqt --non_interactive --force_save command_list --list 'add_hydrogen --add_charges ADT --add_mode auto '



...

2023-04-19 11:38:43,182 [MainThread  ] [INFO ]  /content/drive/MyDrive/MolDock4Colab/Output/prep_receptor.pdbqt file ends with END, cleaning


INFO:/content/log8.out:/content/drive/MyDrive/MolDock4Colab/Output/prep_receptor.pdbqt file ends with END, cleaning


2023-04-19 11:38:43,203 [MainThread  ] [INFO ]  Removed: ['/content/94a5f142-c6d7-4b60-a3f8-5cf5a45d6ca7']


INFO:/content/log8.out:Removed: ['/content/94a5f142-c6d7-4b60-a3f8-5cf5a45d6ca7']


0

## 1.3. Find Binding Cavity

Molecular Docking Simulations are usually restricted to a specific region of the protein's surface, the binding pocket. It we considered the region as the whole surface, we would be performing what is called Blind Docking. 


Some options for finding binding cavities:
- Cluster90.  The PDB Cluster90 is a collection derived from the PDB database, containing structures having less than 90% sequence identity to each other. The Cluster90 Binding Site is using information from all the structures of the Cluster90 collection for a particular input protein to discover the possible binding sites from small molecules attached to similar proteins. 

- [PDB REST API](https://www.ebi.ac.uk/pdbe/api/doc/).  This REST-API endpoint provides details on binding sites from the PDB files (or mmcif) information, such as ligands, residues in the site or description of the site.
- [FindPocket](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-10-168).  pocket is a protein pocket prediction algorithm. Given a PDB protein structure it enables the user to identify potent binding sites. Based on Voronoi tessellation, this algorithm is very fast and particularly well suited for large scale protein binding pocket screenings and development of scoring functions for binding pocket characterization. 

\\

In this particular example, we already know the binding site region. And we can use SMINA direcly with crystalographic ligand as spacial reference for our simulation. But let us consider that we do not have this information. 
One posible option, as it was stated, is the use of the PDBe REST-API, to idenfity and extract the possible binding sites of our taget protein from the PDB file annotations. 

In the following chunk we use the PDBe REST-API by means of the biobb_io package to retrieve information about the residues in the biding pocket.

In [21]:
from biobb_io.api.api_binding_site import api_binding_site

residues_json = os.path.join(PATH, "residues.json")
prop = {
        "pdb_code": pdb_code
}

api_binding_site(output_json_path=residues_json,
                 properties=prop)

### checking output:
import json

with open(residues_json) as json_file: 
    data = json.load(json_file)
    #print(json.dumps(data, indent=4))
    
print('=='*2)
print(f'Number of elements in dic: {len(data)}')
print(f'Prot retrieved: {list(data.keys())}')
print(f'PDB CODE: {pdb_code}')

2023-04-19 11:47:11,438 [MainThread  ] [INFO ]  Executing biobb_io.api.api_binding_site Version: 3.8.0


INFO:/content/log9.out:Executing biobb_io.api.api_binding_site Version: 3.8.0


2023-04-19 11:47:11,447 [MainThread  ] [INFO ]  Getting binding sites from: https://www.ebi.ac.uk/pdbe/api/pdb/entry/binding_sites/2brc


INFO:/content/log9.out:Getting binding sites from: https://www.ebi.ac.uk/pdbe/api/pdb/entry/binding_sites/2brc


2023-04-19 11:47:12,330 [MainThread  ] [INFO ]  1 binding sites found


INFO:/content/log9.out:1 binding sites found


2023-04-19 11:47:12,335 [MainThread  ] [INFO ]  Writting json to: /content/drive/MyDrive/MolDock4Colab/Output/residues.json


INFO:/content/log9.out:Writting json to: /content/drive/MyDrive/MolDock4Colab/Output/residues.json


====
Number of elements in dic: 1
Prot retrieved: ['2brc']
PDB CODE: 2BRC


It found exactly our protein, and returned the info of the binding pocket.
Uncomment the following line to see how the dictionary looks like. 

In [ ]:
#data # uncoment to take a look to see what its inside the dictionary

Now we need to select the binding site, in this case we know is the first one.

In [22]:
bindingSites = {}
bsites = []
for i, item in enumerate(data[pdb_code.lower()]):
    bindingSites[i] = []
    bsites.append(('binding_site' + str(i), i))
    for res in item['site_residues']:
        bindingSites[i].append(res['author_residue_number'])
        
# this is for model selection
# wwe dont actualy need this
# bindingSites[mdsel.value] can be changed for bindingSites[0]

mdsel = ipywidgets.Dropdown(
    options=bsites,
    description='Binding Site:',
    disabled=False,
)
display(mdsel)

A Jupyter Widget

In [23]:
# we save the data in a npy
import numpy as np

bindingsite_data = os.path.join(PATH, 'bindingsite.npy')
np.save(bindingsite_data, bindingSites[mdsel.value])

We can visualize now:


In [24]:
view = nv.show_structure_file(download_pdb, default=False)

view.add_representation(repr_type='cartoon', 
                        selection='not het',
                        opacity=.2,
                        color='#cccccc')

view.add_representation(repr_type='surface', 
                        selection=', '.join(str(v) for v in bindingSites[mdsel.value]), 
                        color='pink',
                        lowResolution= True,
                        # 0: low resolution 
                        smooth=1 )

view.center()
view._remote_call('setSize', target='Widget', args=['','400px'])

view

A Jupyter Widget

We can also retrieve the box as a pdb file by running the next chunk. You can also visualize all toghether in pymol the box, the protein and the crystalographic ligand. 

In [25]:
from biobb_vs.utils.box_residues import box_residues

output_box = os.path.join(PATH, "box.pdb")

prop = {
        "resid_list": bindingSites[mdsel.value],
        "offset": 5,
        "box_coordinates": True
}

box_residues(input_pdb_path = download_pdb,
            output_pdb_path = output_box,
            properties=prop)

2023-04-19 11:47:58,201 [MainThread  ] [INFO ]  Executing biobb_vs.utils.box_residues Version: 4.0.0


INFO:/content/log10.out:Executing biobb_vs.utils.box_residues Version: 4.0.0


2023-04-19 11:47:58,215 [MainThread  ] [INFO ]  Copy: /content/drive/MyDrive/MolDock4Colab/Output/download.pdb to /content/c2e472ff-7a61-409a-bfbc-d08ce616a9de


INFO:/content/log10.out:Copy: /content/drive/MyDrive/MolDock4Colab/Output/download.pdb to /content/c2e472ff-7a61-409a-bfbc-d08ce616a9de


2023-04-19 11:47:58,218 [MainThread  ] [INFO ]  Loading input PDB structure /content/drive/MyDrive/MolDock4Colab/Output/download.pdb


...

2023-04-19 11:47:58,301 [MainThread  ] [INFO ]  Saving output PDB file (with box setting annotations): /content/drive/MyDrive/MolDock4Colab/Output/box.pdb


INFO:/content/log10.out:Saving output PDB file (with box setting annotations): /content/drive/MyDrive/MolDock4Colab/Output/box.pdb


2023-04-19 11:47:58,306 [MainThread  ] [INFO ]  Removed: ['/content/c2e472ff-7a61-409a-bfbc-d08ce616a9de']


INFO:/content/log10.out:Removed: ['/content/c2e472ff-7a61-409a-bfbc-d08ce616a9de']


0

...

In [26]:
## May noy work correcly in colab, if not visualice in pymol 

view = nv.NGLWidget()

s = view.add_component(download_pdb)
b = view.add_component(output_box)

atomPair = [
    [ "9999:Z.ZN1", "9999:Z.ZN2" ],
    [ "9999:Z.ZN2", "9999:Z.ZN4" ],
    [ "9999:Z.ZN4", "9999:Z.ZN3" ],
    [ "9999:Z.ZN3", "9999:Z.ZN1" ],
    
    [ "9999:Z.ZN5", "9999:Z.ZN6" ],
    [ "9999:Z.ZN6", "9999:Z.ZN8" ],
    [ "9999:Z.ZN8", "9999:Z.ZN7" ],
    [ "9999:Z.ZN7", "9999:Z.ZN5" ],
    
    [ "9999:Z.ZN1", "9999:Z.ZN5" ],
    [ "9999:Z.ZN2", "9999:Z.ZN6" ],
    [ "9999:Z.ZN3", "9999:Z.ZN7" ],
    [ "9999:Z.ZN4", "9999:Z.ZN8" ]
]

# structure
s.add_representation(repr_type='cartoon', 
                        selection='not het',
                        color='#cccccc',
                       opacity=.2)
# ligands box
b.add_representation(repr_type='ball+stick',
                     selection='9999',
                     color='pink', 
                     aspectRatio = 8)
# lines box
b.add_representation(repr_type='distance', 
                     atomPair= atomPair,
                     labelVisible=False,
                     color= 'black')

# residues
s.add_representation(repr_type='surface', 
                        selection=', '.join(str(v) for v in bindingSites[mdsel.value]), 
                        color='skyblue',
                        lowResolution= True,
                        # 0: low resolution 
                        smooth=2,
                        surfaceType= 'av', 
                        contour=True,
                        opacity=1,
                        useWorker= True,
                        wrap= True)


view.center()
view._remote_call('setSize', target='Widget', args=['','400px'])

view

A Jupyter Widget

Now, we have all the needed informtion for our receptor. 

In [31]:
# to keep clean your directory, you can run this to delete all residual files. 
! rm *.out
! rm *.err

rm: cannot remove '*.out': No such file or directory
rm: cannot remove '*.err': No such file or directory
